# Demonstrating matrix multiplication with sparse python matrices

In [56]:
# Import necessary modules
from scipy.sparse import csr_matrix, triu, spdiags
import numpy as np
import matplotlib as plt

Create the matrix 
$$A = \begin{bmatrix} 1 & 1 & 1 & 1 \\ 0 & 1 & 1 & 1\\ 0 & 0 & 1 & 1\\ 0 & 0 & 0 & 1 \end{bmatrix}$$

which is an upper triangular matrix of 1s. We will do the matrix multiplacion $$Ax$$ where $$x = [2, 3, 4, 5]^\intercal$$. We will see two ways to do this.

In [57]:
# Create the matrix 
e = np.ones((4,4))
A = spdiags(e, [0, 1, 2, 3], 4, 4, format='csc') # Use "format='csc'" to make solving faster
x = [2, 3, 4, 5]

Method 1

In [58]:
print(A@x)

[14. 12.  9.  5.]


Method 2

In [59]:
print(A.dot(x))

[14. 12.  9.  5.]


Both methods are the same, but maybe one is more efficient? Let's see how long it takes to do a large matrix multiply. 

In [60]:
import time # For timing

In [61]:
N = int(1e4) # Choose N really big
x = np.random.rand(N)
e = np.random.rand(N, N)
A = spdiags(e, np.linspace(0, N, num=N, dtype='int'), N, N, format='csc')

In [62]:
A.toarray()

array([[0.08101436, 0.68629637, 0.73977078, ..., 0.18860616, 0.15509993,
        0.        ],
       [0.        , 0.45307105, 0.21630491, ..., 0.78811847, 0.22998014,
        0.77071414],
       [0.        , 0.        , 0.00486774, ..., 0.06971102, 0.94616428,
        0.05101785],
       ...,
       [0.        , 0.        , 0.        , ..., 0.34776915, 0.23871291,
        0.30135511],
       [0.        , 0.        , 0.        , ..., 0.        , 0.81182711,
        0.44765061],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.15422389]])

First time `A.dot(x)`

In [63]:
start = time.time()
A.dot(x)
print(time.time()-start)

0.14868927001953125


Compare it to `A@x`.

In [64]:
start = time.time()
A@(x)
print(time.time()-start)

0.08609700202941895


We see that both have similar time. Out of curiosity let's check: what if we did this same multiplation with a nonsparse matrix?

In [65]:
B = A.toarray()

In [66]:
start = time.time()
B@(x)
print(time.time()-start)

3.741682291030884


We can see that this is a lot slower.

## Elementwise multiplication

First consider multiplying every column of the matrix $A$ by a number in an array.

Let's start with our old example.

In [67]:
e = np.ones((4,4))
A = spdiags(e, [0, 1, 2, 3], 4, 4)
A.todense()
x = np.array([2, 3, 4, 5])

In [68]:
A.toarray()

array([[1., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.]])

In [69]:
A.multiply(x)

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in COOrdinate format>

In [44]:
A.multiply(x).toarray()

array([[2., 3., 4., 5.],
       [0., 3., 4., 5.],
       [0., 0., 4., 5.],
       [0., 0., 0., 5.]])

`A.multiply(x)` multiplies colums by the array x. 

To multiply rows by the array `x`, we need to reshape `x` to be a column vector. 

In [45]:
y = x.reshape(-1, 1)

In [46]:
A.multiply(y).toarray()

array([[2., 2., 2., 2.],
       [0., 3., 3., 3.],
       [0., 0., 4., 4.],
       [0., 0., 0., 5.]])

We can see that this multiplies each row of the matrix by the corresponding element in the column vector `x`.

# Demonstrating using LU decomposition for sparse matrices.

If you have sparse matrices, you should use `scipy.sparse.linalg.splu` to solve quickly.

Let's create a really big `A` again to see how this works and time it.

In [47]:
N = int(1e4) # Choose N really big
b = np.random.rand(N)
e = np.random.rand(N, N)
A = spdiags(e, np.linspace(0, N, num=N, dtype='int'), N, N, format='csc')
N

10000

In [52]:
A.toarray()[0:9,0:9]

array([[0.32753022, 0.77688265, 0.50131839, 0.76193937, 0.85699886,
        0.07304958, 0.44269547, 0.75117575, 0.50157078],
       [0.        , 0.89943727, 0.2969741 , 0.62816537, 0.912055  ,
        0.59687116, 0.81727958, 0.48610801, 0.93822761],
       [0.        , 0.        , 0.07287084, 0.02798827, 0.085559  ,
        0.60256905, 0.32095581, 0.87088685, 0.24301629],
       [0.        , 0.        , 0.        , 0.05325078, 0.82121908,
        0.24290892, 0.4322692 , 0.467141  , 0.63928451],
       [0.        , 0.        , 0.        , 0.        , 0.63194497,
        0.22021112, 0.74107796, 0.40388737, 0.29050827],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.384187  , 0.64051608, 0.90265014, 0.97106841],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.53784907, 0.87624401, 0.60417584],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.80623198, 0.36312058],


Let's also import some libraries for solving sparse systems.

In [53]:
from scipy.sparse.linalg import spsolve, splu

First, we could just solve Ax = b.

In [54]:
start = time.time()
spsolve(A,b)
print(time.time()-start)

4.37162709236145


But what if we did LU decomposition **first**? To do that, use `splu`.

In [27]:
PLU = splu(A)
start = time.time()
PLU.solve(b)
print(time.time()-start)

0.11018586158752441


It's way faster!